# Ghouls, Goblins, and Ghosts
## Multiclass Classification Task

Falconi Nicasio

April 23 2019

In [6]:
# load data
import pandas as pd
train_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")

In [7]:
from sklearn.preprocessing import LabelEncoder

# encode color features
gle = LabelEncoder()
train_color_labels = gle.fit_transform(train_data['color'])
test_color_labels = gle.fit_transform(test_data['color'])
train_data['color_labels'] = train_color_labels
test_data['color_labels'] = test_color_labels

In [8]:
# take target out of training set
Y = train_data['type']
train_data = train_data.drop(['type', 'id', 'color'], axis=1)
test_data = test_data.drop(['id', 'color'], axis=1)

In [9]:
# standardize values
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()

In [10]:
train_data.describe()

,bone_length,rotting_flesh,hair_length,has_soul,color_labels
count,3.710000e+02,3.710000e+02,3.710000e+02,3.710000e+02,3.710000e+02
mean,4.387027e-16,-5.284781e-16,-9.288766e-16,2.503538e-15,-4.488772e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.809001e+00,-2.809293e+00,-2.322011e+00,-2.623018e+00,-2.107595e+00
25%,-7.088173e-01,-6.288435e-01,-7.162151e-01,-7.005625e-01,-2.503082e-01
50%,5.504366e-03,-3.619028e-02,5.608044e-02,-2.850387e-02,-2.503082e-01
75%,6.253172e-01,6.636377e-01,6.952850e-01,7.336534e-01,9.878831e-01
max,2.882125e+00,2.908063e+00,2.771516e+00,2.636297e+00,9.878831e-01


In [11]:
#check correlation for possible feature engineering
train_data.corr()

,bone_length,rotting_flesh,hair_length,has_soul,color_labels
bone_length,1.000000,-0.041716,0.353881,0.381675,-0.033830
rotting_flesh,-0.041716,1.000000,-0.220353,-0.132051,-0.041714
hair_length,0.353881,-0.220353,1.000000,0.474835,0.009093
has_soul,0.381675,-0.132051,0.474835,1.000000,-0.025546
color_labels,-0.033830,-0.041714,0.009093,-0.025546,1.000000


In [12]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# initiate model
model_to_set = OneVsRestClassifier(LogisticRegression(penalty = 'l2', max_iter = 1000))

# create parameter options
parameters = {
    "estimator__C": [0.1, 0.5, 0.7, 1.0, 1.2, 2, 5, 10, 20, 100],
    "estimator__solver": ['newton-cg', 'lbfgs', 'sag'],
    "estimator__multi_class" : ['multinomial', 'ovr']
}

# Cross validation
model_tunning = GridSearchCV(model_to_set, param_grid=parameters,
                             cv = 2)

model_tunning.fit(train_data, Y)

print(model_tunning.best_score_)
print(model_tunning.best_params_)

0.738544474393531
{'estimator__C': 0.1, 'estimator__multi_class': 'ovr', 'estimator__solver': 'newton-cg'}


In [13]:
# create model
ovr = OneVsRestClassifier(LogisticRegression(penalty = "l2", C = 0.1, multi_class = 'ovr', solver = 'newton-cg'))
# train model
ovr_fitted = ovr.fit(train_data, Y)
# predict test_data
res = ovr_fitted.predict(test_data)

In [14]:
# save predictions
sample_data = pd.read_csv("../input/sample_submission.csv")
sample_data['type'] = res
sample_data.to_csv('prediction.csv', index = False)